In [1]:

############### Crescimento do M2 Brasileiro ###############

# Importa as bibliotecas

from bcb import sgs
import datetime
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import pandas as pd
import numpy as np

# Escolhendo o período
end_date = datetime.date.today()
start_date = '2010-01-01'

# Acesso aos dados pelo pacote do BACEN
m2 = sgs.get({'m2': '27810'}, start=start_date)

# Criar o gráfico principal
plt.figure(figsize=(16, 8))
plt.title("Crescimento do M2 - Base de dados: SGS-BACEN", fontsize=15)
plt.plot(m2.index, m2["m2"], label="M2", color="blue")

# Adicionar grid ao gráfico
plt.grid(True, linestyle="--", alpha=0.5)

# Configurar o eixo Y
plt.ylabel("M2 (em bilhões)")

# Função para formatar o eixo Y em milhares/milhões/bilhões
def format_y(value, _):
    if value >= 1e9:
        return f'{value/1e9:.1f}B'  # Bilhões
    elif value >= 1e6:
        return f'{value/1e6:.1f}M'  # Milhões
    elif value >= 1e3:
        return f'{value/1e3:.1f}K'  # Milhares
    return f'{value:.0f}'  # Valor normal

# Aplicar a formatação ao eixo Y
plt.gca().yaxis.set_major_formatter(mticker.FuncFormatter(format_y))

# Rotacionar os rótulos do eixo X para melhor visualização
plt.xticks(rotation=45)

#---------------Cálculo das Regressão do período-------------------------#

# Separar os períodos antes e depois de 2020
m2_ate_2020 = m2[m2.index < "2020-01-01"]
m2_depois_2020 = m2[m2.index >= "2020-01-01"]

# Converter datas para valores numéricos
x_ate_2020 = (m2_ate_2020.index - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')
x_depois_2020 = (m2_depois_2020.index - pd.Timestamp("1970-01-01")) // pd.Timedelta('1D')

# Ajustar uma reta de regressão para cada período
coef_ate_2020 = np.polyfit(x_ate_2020, m2_ate_2020["m2"], 1)
coef_depois_2020 = np.polyfit(x_depois_2020, m2_depois_2020["m2"], 1)

# Criar as equações da reta
regressao_ate_2020 = np.poly1d(coef_ate_2020)
regressao_depois_2020 = np.poly1d(coef_depois_2020)

# Plotar as retas de regressão
plt.plot(m2_ate_2020.index, regressao_ate_2020(x_ate_2020), color="purple", linestyle="--", label="Regressão até 2020")
plt.plot(m2_depois_2020.index, regressao_depois_2020(x_depois_2020), color="red", linestyle="--", label="Regressão após 2020")

#---------------Cálculo das Regressão do período-------------------------#

#---------------Cálculo dos CAGR período-------------------------#

# Obter os valores iniciais e finais de cada período
V_i_ate_2020 = m2_ate_2020["m2"].iloc[0]
V_f_ate_2020 = m2_ate_2020["m2"].iloc[-1]
n_ate_2020 = (m2_ate_2020.index[-1] - m2_ate_2020.index[0]).days / 365.25

V_i_depois_2020 = m2_depois_2020["m2"].iloc[0]
V_f_depois_2020 = m2_depois_2020["m2"].iloc[-1]
n_depois_2020 = (m2_depois_2020.index[-1] - m2_depois_2020.index[0]).days / 365.25

# Calcular o CAGR (Crescimento Médio Anual)
cagr_ate_2020 = ((V_f_ate_2020 / V_i_ate_2020) ** (1 / n_ate_2020)) - 1
cagr_depois_2020 = ((V_f_depois_2020 / V_i_depois_2020) ** (1 / n_depois_2020)) - 1

# Exibir os valores no gráfico
x_texto_1 = m2_ate_2020.index[int(len(m2_ate_2020) * 0.7)]  # Posição 80% do período
x_texto_2 = m2_depois_2020.index[int(len(m2_depois_2020) * 0.7)]  # Posição 20% do período
y_pos_ate_2020 = m2_ate_2020["m2"].max() * 0.6  # 90% da altura máxima
y_pos_depois_2020 = m2_depois_2020["m2"].max() * 0.7

plt.text(x_texto_1, y_pos_ate_2020, f"CAGR até 2020:\n{cagr_ate_2020 * 100:.2f}%", fontsize=12, color="purple")
plt.text(x_texto_2, y_pos_depois_2020, f"CAGR após 2020:\n{cagr_depois_2020 * 100:.2f}%", fontsize=12, color="red")

#---------------Cálculo dos CAGR período-------------------------#

plt.axvline(pd.Timestamp("2020-01-01"), color="gray", linestyle="--", alpha=0.7, label="Início de 2020")

# Adicionar legenda
plt.legend()

# Exibir o gráfico
plt.show()
